<b><font size=20, color='#A020F0'>Class Notebook 4

## Welcome to your fourth in-class jupyter notebook activity!
In this notebook you'll get more practice with xarray. We'll be using data from the [World Ocean Atlas 2018](https://www.ncei.noaa.gov/products/world-ocean-atlas) (WOA18), which is a global gridded data product derived from oceanographic data from the [World Ocean Database](https://www.ncei.noaa.gov/products/world-ocean-database). And yes, I _know_ it's more oceanography this time, but your homework uses ERA5 data, and next week I think we'll use some sea ice data and data from MERRA2 (no promises though) :)

---

<b><font color='red'>Due Date: 15 October 2021</font></b><br>(by the end of class)

<b>How you will turn in this assignment</b><br> When you are ready to turn in your notebook, do the following:
1. Save your notebook and make sure your results are visible (cells have been executed)
2. Open a terminal and navigate to your local `AOS573` repository
3. If you haven't done so, move your completed in-class notebook into your InClass directory (if you opened and worked on the notebook from this directory you don't need to do anything).
4. Use `git add` and `git commit -m` to add and commit ClassNotebook4 to your local `AOS573` git repository (<font color='red'>_DO NOT ADD THE WOA18 NETCDF FILE!_</font>)
5. `push` your changes to your remote `aos573_completed_assignments` repository: `git push finished_work main` (you'll need to enter your username and personal access token)
6. <b><font color='red'><u>DELETE YOUR LOCAL COPY OF THE WOA18 DATA!

---

## 1. Reading in and exploring your data
The data for this notebook is in the following directory: `/tornado/home1/class/fall21/zanowski/Notebook_data`

Copy the WOA18 data to your home directory (if you put it in your AOS573 directory, <b><font color='red'>_DO NOT_</b></font> add and commit the netCDF file!) <br>

### 1.1 Use xarray to read in the file 'WOA18.nc'

### 1.2 Understanding your data
1. What data variables exist in your dataset and what do they represent? (You can ignore variables ending in _bnds)
2. What are the coordinates of the variables ending in _an?
3. What is the range of values for each coordinate in your dataset?

### 1.3 Visualize your data
Make a 2x2 panel of plots showing temperature, salinity, sigma (sigma=density-1000), and oxygen at the ocean surface. These plots need not be pretty--they should just help you get a sense of what your data actually look like.

### 1.4 Making a land mask
Use np.isnan() to make a land mask and add it to your dataset

---

## 2. Working with your data

### 2.1 Creating basin zonal means
Compute the zonal mean temperature, salinity, and oxygen in the Atlantic, Pacific, and Indian Oceans. 
For simplicity, use the following longitudes for to define your ocean boundaries:
1. Atlantic: 70W-25E
2. Indian:25E-120E
3. Pacific: 120E-70W (290E)

<b>NOTE:</b> Because the Pacific Ocean is split at the dateline, you will either need to use [roll()]((http://xarray.pydata.org/en/stable/generated/xarray.Dataset.roll.html)) to shift your grid so that you you can grab all of the Pacific Ocean data in one continuous chunk, or you can use [concat()](http://xarray.pydata.org/en/stable/generated/xarray.concat.html) to combine the disparate pieces.

### 2.2 Plotting basin zonal means
Make a 3x3 panel of contour plots showing your zonal means for temperature, salinity, and oxygen in each ocean basin. The columns of the plot should be the individual ocean basins, and the rows the variables. Use [annotate()](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.annotate.html) to place a label at the top of each column of plots with the name of the ocean it represents.

### 2.3 Practice interpolating
Use [interp()](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.interp.html) to interpolate global surface ocean salinity onto a new latitude and longitude grid defined as the following:<br>
`lat_new=np.arange(-90.0,90.1,1.0)`<br>
`lon_new=np.arange(-180.0,180.1,1.0)`

Plot your result on top of the original land mask like you did in Q1.5. What happens at the land boundaries (you may need to adjust the x and y limits on your plot to zoom in and see the changes)?

### 2.4 Calculating Ocean Heat Content

#### 2.4.1 Calculate the ocean heat content above 2000 m
Make a quick plot of your result

Ocean Heat Content is defined as the following, where $\rho$=density, $T$=temperature and $C_{p}$ is the specific heat of seawater, and $h$ is an arbitrary depth:<br>
$OHC=C_{p}\int_{0}^{h}\rho(z)T(z)dz$

Some information that will help:
1. $C_{p}$=3990 J/kg/˚C
2. $\rho$=$\sigma$+1000

To help get you started, I've set up the code to compute dz:

In [ ]:
dz=ds.depth_bnds[:,1]-ds.depth_bnds[:,0]

#### 2.4.2 Compute the total ocean heat content above 2000 m by integrating globally
You'll need to compute the area in m$^{2}$ of each grid cell. The radius of the Earth is 6371000 m. The widths and heights of each grid cell are 1˚ in longitude and 1˚ in latitude, respectively. 

### 2.5 Finding the depths of maximum temperature and minimum oxygen
For this problem, find the depth of the maximum temperature and the depth of minimum oxygen at each grid cell (each lat/lon point). Your final result for each should be a 2D field of depths that vary spatially across the planet. I've broken down the problem into a few steps below to help:

#### 2.5.1 Create a 3D DataArray for depth and add it to your dataset
Hint: Try using [full_like()](http://xarray.pydata.org/en/stable/generated/xarray.full_like.html)

#### 2.5.2 Use where() on your 3D depths 
Get the depths where the temperature is a maximum at each grid point and then get the depths where oxygen is a minimum at each grid point (these should be separate fields)

#### 2.5.3 Turn your 3D DataArrays into 2D arrays
In order to be able to make a contour plot of your depths, you need to turn your 3D answer from above into a 2D array with only dimensions of lat and lon. In this case, because each depth column only has one value and the rest are nans, you can do this easly by summing along depth (xarray ignores nans, so your values shouldn't change).

#### 2.5.4 Remask your results so that land is nan again
When you do computations like this, sometimes the nans for land aren't preserved, but this isn't an issue. We just need to remask our 2D results from 2.4.2 by using where(). The land DataArray is 3D, so use only the masked values at the surface (depth=0). Also note that land is a boolean DataArray where the values for the ocean values = False, so your condition in where() statement should be where land is false.

#### 2.5.5 Plot your results
Make a 1x2 set of filled contour plots of your results. What do your results tell you about the depths where temperatures are largest in the polar oceans? Does this seem to make sense based on your plots from 2.2?